In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/my.csv", index_col=None)

In [3]:
# df=df.drop(['land','su_attempted','num_outbound_cmds','is_host_login','urgent','num_failed_logins','su_attempted','num_file_creations','num_shells','srv_diff_host_rate'], axis = 1)


In [4]:
df.head()

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,25,0.17.1,0.03,0.17.2,0.24,0.25,0.26,0.05,0.27,normal
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune
2,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal
3,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune


In [5]:
y = df[df.columns[-1]].values
from collections import Counter
#num = Counter(y)
#n=num.get('normal')
Counter(y)

Counter({'normal': 67342,
         'neptune': 41214,
         'warezclient': 890,
         'ipsweep': 3599,
         'portsweep': 2931,
         'teardrop': 892,
         'nmap': 1493,
         'satan': 3633,
         'smurf': 2646,
         'pod': 201,
         'back': 956,
         'guess_passwd': 53,
         'ftp_write': 8,
         'multihop': 7,
         'rootkit': 10,
         'buffer_overflow': 30,
         'imap': 11,
         'warezmaster': 20,
         'phf': 4,
         'land': 18,
         'loadmodule': 9,
         'spy': 2,
         'perl': 3})

In [6]:
from sklearn.preprocessing import LabelEncoder
encodings = dict()
for c in df.columns:
    #print df[c].dtype
    if df[c].dtype == "object":
        encodings[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings[c]
        df[c] = encodings[c].fit_transform(df[c])

In [7]:
y = df.pop(df.columns[-1]).values
X = df.values

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer([('t', OneHotEncoder(), [1])], remainder='passthrough')

X = onehotencoder.fit_transform(X)


In [9]:
from sklearn.preprocessing import StandardScaler #normalization
X = StandardScaler().fit_transform(X)

In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
principalDfX = pd.DataFrame(data = principalComponents)

In [11]:
principalDfX.head()
print(Counter(y))

Counter({11: 67342, 9: 41214, 17: 3633, 5: 3599, 15: 2931, 18: 2646, 10: 1493, 0: 956, 20: 892, 21: 890, 14: 201, 3: 53, 1: 30, 22: 20, 6: 18, 4: 11, 16: 10, 7: 9, 2: 8, 8: 7, 13: 4, 12: 3, 19: 2})


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(principalDfX, y, test_size=0.2, random_state=42)#random for splitting same data when run again.
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [13]:
from sklearn.svm import SVC

clf = SVC(gamma='auto',decision_function_shape='ovo')
clf.fit(X_train, y_train)

SVC(decision_function_shape='ovo', gamma='auto')

In [14]:
y_pred=clf.predict(X_test)
print(Counter(y_pred))
print(Counter(y_test))

Counter({11: 14871, 9: 8296, 5: 680, 15: 580, 17: 494, 10: 267, 20: 6, 3: 1})
Counter({11: 13386, 9: 8238, 17: 749, 5: 743, 15: 583, 18: 551, 10: 298, 0: 201, 20: 196, 21: 176, 14: 36, 3: 14, 1: 7, 22: 4, 7: 3, 6: 2, 4: 2, 16: 2, 12: 1, 19: 1, 8: 1, 13: 1})


In [15]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
 
results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_test, y_pred))
print('Report : ')
print(classification_report(y_test, y_pred))


Confusion Matrix :
[[    0     0     0     0     0     0     0     0     0     0   201     0
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     1     0     0     0     0     0     5     0
      0     0     0     0     0     0     0     1     0     0]
 [    0     0     0     0     0     0     0     0     1     0    12     0
      0     0     1     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     2     0
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0   633     0     0     0     0    20    68     0
      0     0    22     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     2     0
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     3     0
      0     0     0     0     0     0     0     0     0     0]
 [    0     0     0   

D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
predicted = cross_val_predict(clf, X_train, y_train, cv=skf)
print('Accuracy Score :',accuracy_score(y_train, predicted))
print('Report : ')
print(classification_report(y_train, predicted))


D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy Score : 0.9321273703325164
Report : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       755
           1       0.00      0.00      0.00        23
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00        39
           4       0.00      0.00      0.00         9
           5       0.95      0.87      0.91      2856
           6       0.00      0.00      0.00        16
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00         6
           9       0.99      1.00      0.99     32976
          10       0.91      0.79      0.85      1195
          11       0.90      1.00      0.95     53956
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00       165
          15       0.92      0.89      0.90      2348
          16       0.00      0.00  